# CityBikes

Send a request to CityBikes for the city of your choice. 

In [1]:
import requests
import pandas as pd
import os 

In [2]:
# Step 1: Sending a request to CityBikes using /v2/networks endpoint 
city_name = 'Barcelona'
networks_url = 'http://api.citybik.es/v2/networks' 
response = requests.get(networks_url)
networks_data = response.json()

In [3]:
# Step 2: Find network information for Barcelona
network_info = next((network for network in networks_data['networks'] if network['location']['city'] == city_name), None)

if network_info:
    # Use network ID to retrieve info of bike stations in Barcelona
    network_id = network_info['id']
    api_url = f'http://api.citybik.es/v2/networks/{network_id}'

    response = requests.get(api_url)
    data = response.json()

In [4]:
# Checking retrieved info from Network ID of bike stations
print(type(data))
print(type(data['network']['stations']))
data['network']['stations'][0].keys()

<class 'dict'>
<class 'list'>


dict_keys(['empty_slots', 'extra', 'free_bikes', 'id', 'latitude', 'longitude', 'name', 'timestamp'])

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [5]:
# Parsing through response to get details for the bike stations
data['network']['stations'][15]

{'empty_slots': 27,
 'extra': {'ebikes': 0,
  'has_ebikes': True,
  'normal_bikes': 0,
  'online': True,
  'uid': 305},
 'free_bikes': 0,
 'id': '094eb7b65cdf38f3f7abd6fe76833690',
 'latitude': 41.3875297,
 'longitude': 2.1237141,
 'name': 'AV. DIAGONAL, 680',
 'timestamp': '2023-12-20T22:33:27.541000Z'}

In [6]:
# Step 3: Extracting station information in Barcelona
station_data = []
for station in data['network']['stations']:
    station_info = {
        'latitude': station['latitude'],
        'longitude': station['longitude'],
        'num_bikes': station['free_bikes'],
    }
    station_data.append(station_info)

Put your parsed results into a DataFrame.

In [7]:
# Step 4: Creating a dataframe to store the above results
df = pd.DataFrame(station_data)

In [8]:
# Step 5: Explore created dataframe 
print(df.head())

print(df.describe())

    latitude  longitude  num_bikes
0  41.425364   2.185207          7
1  41.430000   2.190246         22
2  41.443365   2.190630         13
3  41.377532   2.170732          9
4  41.375116   2.152309         11
         latitude   longitude   num_bikes
count  505.000000  505.000000  505.000000
mean    41.399842    2.168088    9.766337
std      0.021075    0.024644    7.112256
min     41.346775    2.109154    0.000000
25%     41.383904    2.149144    4.000000
50%     41.396141    2.170675    9.000000
75%     41.412068    2.187196   15.000000
max     41.462095    2.220691   35.000000


In [9]:
response_data = data['network']['stations']
bike_stations = pd.json_normalize(response_data)

# Dropping unnecessary columns 
columns_to_drop = list(bike_stations.columns[6:])
bike_stations = bike_stations.drop(columns=columns_to_drop)

In [10]:
# Return all rows of bike stations dataframe 
bike_stations.head()

,empty_slots,free_bikes,id,latitude,longitude,name
0,23,7,dedcd93fbe85671e73c79baf58bd7fed,41.425364,2.185207,"C/ CARDENAL TEDESCHINI, 13"
1,3,22,5c10c2dba1c5a8e8d9dcbb251fd8eab7,41.430000,2.190246,"RAMBLA DE L'ONZE DE SETEMBRE, 69"
2,11,13,f1aaf083aaa04498aad975bc45bdc089,41.443365,2.190630,"PG. TORRAS I BAGES, 129"
3,8,9,4a30020e6926a2ba39f88be0f6bba5cb,41.377532,2.170732,"C/ SANT OLEGUER, 2"
4,15,11,e964297fbcd2fc96bb6632e5a530d0df,41.375116,2.152309,"AV. PARAL.LEL, 194"


In [11]:
# Get dimension of the bike stations dataframe 
bike_stations.shape

(505, 6)

In [12]:
# Retrieve info of dataframe
bike_stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   empty_slots  505 non-null    int64  
 1   free_bikes   505 non-null    int64  
 2   id           505 non-null    object 
 3   latitude     505 non-null    float64
 4   longitude    505 non-null    float64
 5   name         505 non-null    object 
dtypes: float64(2), int64(2), object(2)
memory usage: 23.8+ KB


In [16]:
directory = 'C:/Users/Valerie/OneDrive/StatisticalModellingProject/Notebooks'
file_path = 'bike_stations_barcelona.csv'

if not os.path.exists(directory):
    os.makedirs(directory)

In [17]:
#save data to use in another notebook
bike_stations.to_csv(file_path, index=False)